# Analyzing Mobile App Data

This project focuses on analyzing data about fictional Android and iOS apps that are made available on Google Play and in the App Store.

All apps are made free to download and the only source of revenue comes from in-app ads. Therefore, the number of users determines the profit from each app: the more engaging the add, the more people will use the app. 

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [31]:
open_apple = open(r'C:\Users\aii20\OneDrive - University of Exeter\Desktop\dataquest\AppleStore.csv', encoding="utf8")
open_google = open(r'C:\Users\aii20\OneDrive - University of Exeter\Desktop\dataquest\googleplaystore.csv', encoding="utf8")

from csv import reader
read_apple = reader(open_apple)
read_google = reader(open_google)
apple = list(read_apple)
google = list(read_google)


Documentation for the AppleStore dataset can be found here: https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps. In order to better understand what each column means, press **View more** under the **About Dataset** section.

In [30]:
apple_columns = apple[0]
google_columns = google[0]
print(apple_columns)
print(google_columns)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [36]:
del google[10473]

In [37]:
duplicate_apps = []
unique_apps = []

for app in google[1:]:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('These are some of the duplicate apps:', duplicate_apps[:10])

Number of duplicate apps: 1181


These are some of the duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


As we can see, there are 1181 duplicate apps in the Google Play store. In order for a non-random removal of the duplicates, we'll keep the row with the highest number of reviews, as that is the latest version of the app.

In [38]:
reviews_max = {}
for app in google[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

The above code creates a dictionary which stores the highest number of reviews for each app. It uses the app name as the key and the number of reviews as the value. 

In [40]:
print('The dictionary has', len(reviews_max), 'entries')
print('\n')
print('This is the highest number of reviews for Instagram:', reviews_max['Instagram'])

The dictionary has 9659 entries


This is the highest number of reviews for Instagram: 66577446.0


In [41]:
android_clean = []
already_added = []
for app in google[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name) 

The above code creates two empty lists. One will store the cleaned data set and the other the app names. The data set which stores the app names helps us keep track of the apps already added to the cleaned data set, so not to be added again.

In [42]:
print('The cleaned data set has', len(android_clean), 'apps')
print('\n')
print('The first 3 entries from the cleaned data set are:', android_clean[:3])

The cleaned data set has 9659 apps


The first 3 entries from the cleaned data set are: [['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']]


In [43]:
def ascii(string):
    i = 0
    for character in string:
        if ord(character) > 127:
            i += 1
    if i > 3:
        return False
    return True

The above function checks if more than three characters from the app's name are outside the ASCII range. If so, we'll assume the app is not targeted towards english speakers.

In [46]:
google_english = []
apple_english = []
for app in android_clean:
    name = app[0]
    if ascii(name) == True:
        google_english.append(app)

for app in apple[1:]:
    name = app[1]
    if ascii(name) == True:
        apple_english.append(app)

print('Number of apps on the Apple AppStore targeted towards English speakers: ', len(apple_english))
print('Number of apps on the Google Play Store targeted towards English speakers: ', len(google_english))

Number of apps on the Apple AppStore targeted towards English speakers:  6183
Number of apps on the Google Play Store targeted towards English speakers:  9614


In [47]:
free_google = []
free_apple = []
for app in google_english:
    price = app[7]
    if price == '0':
        free_google.append(app)
        
for app in apple_english:
    price = app[4]
    if price == '0.0':
        free_apple.append(app)

print('Number of free apps on the Apple AppStore targeted towards English speakers: ', len(free_apple))
print('Number of free apps on the Google Play Store targeted towards English speakers: ', len(free_google))

Number of free apps on the Apple AppStore targeted towards English speakers:  3222
Number of free apps on the Google Play Store targeted towards English speakers:  8864


As mentioned in the introduction, our goal is to determine the kinds of apps that are likely to attract more users because the number of people using our apps affect our revenue.

To minimize risks and overhead, the validation strategy for an app idea has three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, develop it further.
3. If the app is profitable after six months, build an iOS version of the app and add it to the App Store.

The end goal is to add the app on both Google Play and the App Store, so we need to find app profiles that are successful in both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

In [49]:
def freq_table(dataset, index):
    frequency_table = {}

    for row in dataset:
        freq = row[index]
        if freq in frequency_table:
            frequency_table[freq] += 1
        else:
            frequency_table[freq] = 1

    for value in frequency_table:            
        frequency_table[value] = frequency_table[value]/len(dataset)*100
    return frequency_table

The above function creates a frequency table with frequencies expressed as percentages. The function below sorts the frequency table in descending order.

In [51]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [52]:
print('Frequency table of app genres on the AppStore: ')
print('\n')
display_table(free_apple, 11)

Frequency table of app genres on the AppStore: 


Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In [53]:
print('Frequency table of app genres on the GooglePlay Store: ')
print('\n')
display_table(free_google, 9)

Frequency table of app genres on the GooglePlay Store: 


Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & D

In [54]:
print('Frequency table of app categories on the GooglePlay Store: ')
print('\n')
display_table(free_google, 1)

Frequency table of app categories on the GooglePlay Store: 


FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.710

The most common genres and categories of free English Google apps are Family, Entertainment, Tools, Games and Education. Family represents the most abundant cateogory of app on the Google PlayStore and Tools are the most abundant genre and the third most abundant category. Compared to the iOS apps, there is not such a big difference between the 1st and 2nd most numerous apps. However, there are many more genres for the Google apps compared to the Apple ones.

An entertainment app (including games), or a family-oriented app seem to be the most popular choices of apps on both markets. Due to their abundance they probably have the most number of users as well

In [56]:
freq_table_apple = freq_table(free_apple,11)

In [57]:
for genre in freq_table_apple:
    total = 0
    len_genre = 0
    for app in free_apple:
        genre_app = app[11]
        if genre_app == genre:
            total += float(app[5])
            len_genre += 1
    average_apple = total / len_genre
    print(genre,':',average_apple)
    

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


The above creates a frequency table of the number of ratings for each app category on the AppStore. It seems that Reference and Music apps have the highest numbeer of ratings on average, meaning they have the most users as well. So an app with any of these two profiles would prove most successful, more than the gaming app previously thought of.

In [58]:
freq_table_google = freq_table(free_google, 1)

In [59]:
for category in freq_table_google:
    total = 0
    len_category = 0
    for app in free_google:
        category_app = app[1]
        if category_app == category:
            installs = app[5]
            installs = installs.replace('+', '')
            installs = installs.replace(',', '')
            total += float(installs)
            len_category += 1
    average_google = total / len_category
    print(category,':',average_google)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

From the above frequency table for Google Play app installs we can see that communication and video player apps have the most installs. The third most downloaded category is social apps, which is also very popular with iOS users. 

What needs to be looked at in more detail is the number of apps downloaded for each genre/category for each market. For example, only a couple of apps can have  a lot of installs, dominating their specific genres and making it very hard for a new app of the same genre to compete with them. One might instinctively think Instagram, Facebook and Twitter are examples of such apps in the social app category.

All in all, more research needs to be done in order to find a profitable app that will have enough downloads and generate a good amount of revenue from in-app ads.